In [1]:
!pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

In [41]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.5 MB/s eta 0:00:00


In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score
from tqdm.auto import tqdm
from torch import nn
from time import time

In [92]:
def text2toxicity(text, aggregate=True):
    """
    Calculate toxicity of a text:
    - if aggregate=True → return average toxicity score
    - if aggregate=False → return probabilities for each toxicity class
    """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()

    class_names = [
        "non-toxic",
        "insult",
        "obscene",
        "threat",
        "dangerous"
    ]

    if isinstance(text, str):
        proba = proba[0]  # Один текст → вектор
        if aggregate:
            agg_value = 1 - proba.T[0] * (1 - proba.T[-1])
            return f"Среднее значение токсичности = {agg_value:.4f}"
        else:
            result = []
            for i, name in enumerate(class_names):
                result.append(f"Класс {i} ({name}) = {proba[i]:.4f}")
            return "\n".join(result)
    else:
        # Список текстов
        if aggregate:
            agg_values = 1 - proba[:, 0] * (1 - proba[:, -1])
            return [f"Среднее значение токсичности = {v:.4f}" for v in agg_values]
        else:
            results = []
            for j, probs in enumerate(proba):
                result = [f"Текст {j}:"]
                for i, name in enumerate(class_names):
                    result.append(f"  Класс {i} ({name}) = {probs[i]:.4f}")
                results.append("\n".join(result))
            return results

print(text2toxicity('я люблю нигеров', True))
# 0.9350118728093193

print(text2toxicity('я люблю нигеров', False))
# [0.9715758  0.0180863  0.0045551  0.00189755 0.9331106 ]

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], True))
# [0.93501186 0.04156357]

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], False))
# [[9.7157580e-01 1.8086294e-02 4.5550885e-03 1.8975559e-03 9.3311059e-01]
#  [9.9979788e-01 1.9048342e-04 1.5297388e-04 1.7452303e-04 4.1369814e-02]]


Среднее значение токсичности = 0.8860
Класс 0 (non-toxic) = 0.9965
Класс 1 (insult) = 0.0023
Класс 2 (obscene) = 0.0021
Класс 3 (threat) = 0.0015
Класс 4 (dangerous) = 0.8159
['Среднее значение токсичности = 0.8988', 'Среднее значение токсичности = 0.0217']
['Текст 0:\n  Класс 0 (non-toxic) = 0.9745\n  Класс 1 (insult) = 0.0177\n  Класс 2 (obscene) = 0.0030\n  Класс 3 (threat) = 0.0016\n  Класс 4 (dangerous) = 0.9091', 'Текст 1:\n  Класс 0 (non-toxic) = 0.9998\n  Класс 1 (insult) = 0.0003\n  Класс 2 (obscene) = 0.0001\n  Класс 3 (threat) = 0.0002\n  Класс 4 (dangerous) = 0.0421']


In [88]:
data = pd.read_csv('/content/labeled(1).csv')
data.head(20)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [111]:
print(data['comment'].iloc[3])

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?



In [97]:
print(text2toxicity([data['comment'][9], data['comment'][10]], False))

['Текст 0:\n  Класс 0 (non-toxic) = 0.8774\n  Класс 1 (insult) = 0.0922\n  Класс 2 (obscene) = 0.0050\n  Класс 3 (threat) = 0.0082\n  Класс 4 (dangerous) = 0.7546', 'Текст 1:\n  Класс 0 (non-toxic) = 0.0017\n  Класс 1 (insult) = 0.9896\n  Класс 2 (obscene) = 0.1423\n  Класс 3 (threat) = 0.0112\n  Класс 4 (dangerous) = 0.9158']


In [10]:
data.shape

(14412, 2)

In [15]:
comments = data["comment"].apply(lambda x: tokenizer(x, max_length=64, truncation=True, padding='max_length')).values
labels = data['toxic'].values
X_train, X_valid, y_train, y_valid = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [11]:
class TinyRubertInputs(torch.utils.data.Dataset):
    def __init__(self, encoded_text, y_true):
        super().__init__()
        self.encoded_text = encoded_text
        self.labels = y_true

    def __len__(self):
        return len(self.encoded_text)

    def __getitem__(self, idx):
        # Берем словарь по индексу из массива
        text_dict = self.encoded_text[idx]
        return {
            "input_ids": torch.tensor(text_dict["input_ids"]).long(),
            "attention_mask": torch.tensor(text_dict["attention_mask"]).long(),
            "labels": torch.tensor(self.labels[idx]).long(),
        }

In [51]:
train_dataset = TinyRubertInputs(X_train, y_train)
val_dataset   = TinyRubertInputs(X_valid, y_valid)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [19]:
train_dataset[0]

{'input_ids': tensor([    2,   311,  1781,  1023, 10680, 28475, 25933,   769,  4159,  1186,
         22541,    16,   871, 17717,  5557, 17753,   769, 26337,   938,    13,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(0)}

In [78]:
def evaluate_model(model, dev_dataloader, threshold=0.5):
    model.eval()
    preds = []
    facts = []

    for batch in tqdm(dev_dataloader):
        # Сохраняем истинные метки
        facts.append(batch["labels"].cpu().numpy())  # не делаем ravel пока

        batch = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}

        with torch.no_grad():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch.get("token_type_ids", None)
            )

        probs = torch.sigmoid(outputs.logits).cpu().numpy()
        preds.append(probs)  # не делаем ravel пока

    # Объединяем и проверяем формы
    facts = np.concatenate(facts)
    preds = np.concatenate(preds)

    print(f"Facts shape: {facts.shape}")  # для отладки
    print(f"Preds shape: {preds.shape}")  # для отладки

    # Если shapes не совпадают, нужно выровнять
    if facts.shape != preds.shape:
        # Если это многоклассовая классификация, берем argmax или обрабатываем соответствующим образом
        if len(preds.shape) > 1 and preds.shape[1] > 1:
            # Для многоклассовой классификации
            preds = preds.argmax(axis=1) if preds.shape[1] > 1 else preds.ravel()
            facts = facts.ravel()
        else:
            # Для бинарной классификации
            preds = preds.ravel()
            facts = facts.ravel()

    # Теперь проверяем снова
    if len(facts) != len(preds):
        # Если все еще не совпадают, обрезаем до минимальной длины
        min_length = min(len(facts), len(preds))
        facts = facts[:min_length]
        preds = preds[:min_length]
        print(f"Adjusted to min length: {min_length}")

    preds_bin = (preds >= threshold).astype(int)

    results = {
        "roc_auc": roc_auc_score(facts, preds),
        "f1": f1_score(facts, preds_bin),
        "accuracy": accuracy_score(facts, preds_bin),
        "precision": precision_score(facts, preds_bin),
        "recall": recall_score(facts, preds_bin)
    }

    return results

model.eval()
print(evaluate_model(model, train_loader))

  0%|          | 0/181 [00:00<?, ?it/s]

Facts shape: (11529,)
Preds shape: (11529, 5)
{'roc_auc': np.float64(0.7490015975382008), 'f1': 0.6711906230721777, 'accuracy': 0.8150750281897823, 'precision': 0.8378898729303041, 'recall': 0.5598147671726267}


In [77]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [79]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

In [80]:
gradient_accumulation_steps = 1
window = 500
cleanup_step = 100
report_step = 5000

In [82]:
ewm_loss = 0
model.train()
cleanup()
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='none')

for epoch in trange(15):
    tq = tqdm(train_loader)

    for i, batch in enumerate(tq):
        try:
            batch = {k: v.to(model.device) for k, v in batch.items()}
            output = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch.get("token_type_ids", None)
            )
            loss_by_class = (loss_fn(output.logits, batch["labels"]) * batch["labels_mask"]).mean(dim=0)
            loss = loss_by_class.sum()
            loss.backward()
        except RuntimeError as e:
            print('error on step', i, e)
            loss = None
            cleanup()
            continue

        if i and i % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % cleanup_step == 0:
            cleanup()

        w = 1 / min(i+1, window)
        ewm_loss = ewm_loss * (1-w) + loss.item() * w
        tq.set_description(f'loss: {ewm_loss:4.4f}')

        if i % report_step == 0:
            model.eval()
            eval_loss = evaluate_model(model, dev_dataloader)
            model.train()
            print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/181 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([64, 5]))